# Манипуляции с файлами Excel (.xlsx)

Посмотрим на манипуляции, которые можно производить с таблицами Excel программно. Анализ табличных данных мы будем делать с помощью pandas. Но часто для отчетов необходимо собрать дашборд или обновить текущие таблицы с форматированием. Все это можно сделать с помощью библиотеки openpyxl.

In [31]:
import openpyxl
from openpyxl import Workbook
from openpyxl.chart import BarChart, LineChart, Reference
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.formatting.rule import ColorScaleRule, IconSet, FormatObject, Rule, DataBarRule, CellIsRule, FormulaRule

### Столбчатая диаграмма

Загрузим файл Excel с именем "sample.xlsx", выберем лист с именем "bar", создадим столбчатую диаграмму из указанного диапазона ячеек, зададим заголовки для осей диаграммы, а также добавим диаграмму на лист в указанном месте. После всех операций файл сохраняем и закрываем.

In [55]:
workbook = openpyxl.load_workbook('sample.xlsx') # Загрузка файла Excel
sheet = workbook['bar'] # Выбор листа "bar" в файле

bar_chart = BarChart() # Создание объекта столбчатой диаграммы

bar_chart.type = "col" # Установка типа диаграммы на столбчатую
bar_chart.style = 15 # Установка стиля диаграммы из пресетов Excel (от 10 до 15)

bar_chart.title = "Столбчатая диаграмма" # Установка заголовка для диаграммы
bar_chart.y_axis.title = 'Количество покупок' # Установка заголовка для оси Y (вертикальной)
bar_chart.x_axis.title = 'Категория покупок' # Установка заголовка для оси X (горизонтальной)

# Создание ссылок на данные для осей X и Y диаграммы
data = Reference(sheet, min_col = 2, min_row = 2, max_row = 7)
cats = Reference(sheet, min_col = 1, min_row = 2, max_row = 7)

# Добавление данных на диаграмму и отключение использования заголовков данных и легенды
bar_chart.add_data(data, titles_from_data=False)
bar_chart.set_categories(cats)
bar_chart.legend = None

# Добавление диаграммы на лист Excel, якорь указывается как 'D1'
sheet.add_chart(bar_chart, anchor='D1')

workbook.save("sample.xlsx") # Сохранение изменений в рабочей книге
workbook.close() # Закрытие рабочей книги

### Линейная диаграмма

Работает аналогично варианту выше, только создается через другой класс LineChart().

In [18]:
workbook = openpyxl.load_workbook('sample.xlsx') # Загрузка файла Excel
sheet2 = workbook['line'] # Выбор листа "line" в файле

line_chart = LineChart() # Создание объекта линейной диаграммы

# Установка заголовков как в примере выше
line_chart.title = 'Линейная диаграмма'
line_chart.y_axis.title = 'Количество купленных карандашей'
line_chart.x_axis.title = 'Дата'

# Создание ссылок на данные для осей X и Y диаграммы
values = Reference(sheet2, min_col = 2, min_row = 2, max_row = 7)
dates = Reference(sheet2, min_col = 1, min_row = 2, max_row = 7)

# Добавление данных на диаграмму и отключение использования заголовков данных и легенды
line_chart.add_data(values)
line_chart.set_categories(dates)
line_chart.legend = None

# Добавление диаграммы на лист Excel с якорем
sheet2.add_chart(line_chart, anchor='D1')

workbook.save("sample.xlsx") # Сохранение изменений в рабочей книге
workbook.close() # Закрытие рабочей книги

### Изменение стиля форматирования

Для каждой ячейки в диапазоне A2:D5 применяет определенные стили: заполнение цветом, границы, выравнивание текста, а также устанавливает высоту строк и ширину столбцов.

In [58]:
workbook = openpyxl.load_workbook('sample.xlsx')
sheet3 = workbook['style']

# Цикл по строкам и столбцам для применения стилей к ячейкам
for row in range(2, 6):
    for col in range(1, 4):
        # Получение текущей ячейки
        current_cell = sheet3.cell(row, col)
        
        # Заполнение ячейки цветом
        # Цвета в формате HEX (просто загуглите hex color, и вам выпадет палетка с подбором цветов)
        current_cell.fill = PatternFill(fill_type = 'solid',
                                                   start_color = 'c800ff')
        
        # Установка границ ячейки
        current_cell.border = Border(left=Side(border_style = 'double', color = 'FF0000'),
                                                right = Side(border_style = 'double', color = 'FF0000'),
                                                top = Side(border_style = 'double', color = 'FF0000'),
                                                bottom = Side(border_style = 'double', color = 'FF0000'))
        
        # Выравнивание текста в ячейке
        current_cell.alignment = Alignment(wrapText = True, horizontal = 'center', vertical = 'center')
        
        sheet3.row_dimensions[row].height = 80 # Установка высоты строки
        col_letter = get_column_letter(col) # Получение буквенного обозначения столбца
        sheet3.column_dimensions[col_letter].width = 20  # Установка ширины столбца
        
workbook.save("sample.xlsx")
workbook.close()

### Условное форматирование

Все форматирования Excel можно задавать через код. Вот некоторые примеры. Для них мы будем сразу создавать данные (простые списки) и записывать их в ячейки. Поможет нам в этом `enumerate()`.

In [60]:
numbers = [1,2,3,4,5,6,7,8,9,10]
list(enumerate(numbers))

[(0, 1),
 (1, 2),
 (2, 3),
 (3, 4),
 (4, 5),
 (5, 6),
 (6, 7),
 (7, 8),
 (8, 9),
 (9, 10)]

In [61]:
workbook = openpyxl.load_workbook('sample.xlsx')
sheet4 = workbook['format']

# Установка ширины столбцов
for i in ['A', 'B', 'C', 'D', 'E']:
    sheet4.column_dimensions[i].width = 15

### ----- Цветовой градиент ----- ###

sheet4['A1'] = 'Цветовой градиент' # Запись заголовка в ячейку A1

numbers = [1,2,3,4,5,6,7,8,9,10]

# Проход по списку чисел и запись каждого числа в соответствующую ячейку
# index + 2 потому что enumerate начинает индекс с 0 (+1) плюс пропускаем ячейку с заголовком (еще +1)
for index, value in enumerate(numbers):
    sheet4.cell(row = index + 2, column = 1, value = value)
    
# Создание правила для условно форматирования с использованием цветовой шкалы
rule =  ColorScaleRule(start_type='num', start_value = 1, start_color = 'CCFFCC',
                       mid_type = 'num', mid_value = 5, mid_color = '66FF66',
                       end_type = 'num', end_value = 10, end_color = '009900')
    
# Добавление условного форматирования на диапазон ячеек A1:A11 с использованием цветовой шкалы    
sheet4.conditional_formatting.add('A1:A11', rule)  

### ----- Иконки ----- ###

sheet4['B1'] = 'Иконки'

numbers = list(range(-5, 5))

for index, value in enumerate(numbers):
    sheet4.cell(row = index + 2, column = 2, value = value)

# Создание объектов форматирования для определения значений условного форматирования
first = FormatObject(type = 'num', val = -5) # первый интервал значений (для стрелок вниз)
second = FormatObject(type = 'num', val = 0) # второй интервал значений (для стрелки вбок)
third = FormatObject(type = 'num', val = 1) # третий интервал значений (для стрелки вверх)

# Создание объекта набора иконок
iconset = IconSet(iconSet = '3Arrows', cfvo = [first, second, third])

# Создание правила для условного форматирования типа "iconSet" с использованием созданного набора иконок
rule = Rule(type = 'iconSet', iconSet = iconset)

# Добавление условного форматирования на диапазон ячеек B1:B11 с использованием заданного правила
sheet4.conditional_formatting.add('B1:B11', rule)


### ----- Спарклайны ----- ###

sheet4['C1'] = 'Спарклайны'

numbers = list(range(1, 11))

for index, value in enumerate(numbers):
    sheet4.cell(row = index + 2, column = 3, value = value)
    
# Создание правила для условного форматирования типа "Data Bar" (спарклайн); цвет из правил Excel
rule = DataBarRule(start_type = 'num', start_value = 1, end_type = 'num', end_value = 10, color = 'FF638EC6')

# Добавление условного форматирования на диапазон ячеек C1:C11 с использованием заданного правила
sheet4.conditional_formatting.add('C1:C11', rule)


### ----- Форматирование по условию ----- ###

sheet4['D1'] = 'Равенство'

# Создание заливки красным цветом
redFill = PatternFill(start_color = 'EE1111', end_color = 'EE1111', fill_type = 'solid')

numbers = [5, 25, 5, 12, 1, -5, 5, 8, 55, 5]

for index, value in enumerate(numbers):
    sheet4.cell(row = index + 2, column = 4, value = value)

# Создание правила для условного форматирования типа "Cell Is" (ячейка равна) и добавление на диапазон D1:D11
sheet4.conditional_formatting.add('D1:D11', CellIsRule(operator='equal', formula=['5'], fill=redFill))

### ----- Форматирование по условию ----- ###
sheet4['E1'] = 'Четные числа'

# Создание заливки желтым цветом
yellowFill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')

numbers = list(range(1, 11))

for index, value in enumerate(numbers):
    sheet4.cell(row = index + 2, column = 5, value = value)

# Создание правила для условного форматирования типа "Formula" (формула)
# Формула ISEVEN(E1) проверяет, является ли значение в ячейке E1 четным
# Добавление условного форматирования на диапазон ячеек E1:E11
sheet4.conditional_formatting.add('E1:E11', FormulaRule(formula=['ISEVEN(E1)'], fill=yellowFill))


# Проход по списку строк с ячейками заголовков столбцов
for c in ['A1', 'B1', 'C1', 'D1', 'E1']:
    
    # Установка жирного шрифта для текста в ячейке
    sheet4[c].font = Font(bold=True) 
    
    # Выравнивание текста по центру ячейки и перенос текста
    sheet4[c].alignment = Alignment(wrapText=True, horizontal='center', vertical='center') 
       
workbook.save("sample.xlsx")
workbook.close()